In [2]:
# 署名プログラム(秘密鍵）

import sys
sys.set_int_max_str_digits(32768)
import time
import pickle
import random
from random import randint
from fractions import Fraction

am = []
act = [] # 全暗号文
abl = 0 # 総桁数(10進)

# 正の成分を持つ定ベクトル　dv（公開）
dv1 = Fraction(1000) # dvの第1成分（公開）
dv2 = Fraction(1000) # dvの第2成分（公開）

# 0 128bit-sec. or 256bit-sec.
num0 = int(input("Select number 1 or 2 (1:128bit-sec.,  2:256bit-sec.): "))

# 1 decimal of plaintext
num1 = int(input("Select number 1 or 2 or 3 (1:40decimal(128bit), 2:80decimal(256bit), 3:160decimal(512bit)): ")) 

start = time.perf_counter() # 現在時刻（処理開始前）を取得

for i in range(1): # choice of plaintext
    #start = time.perf_counter() # 現在時刻（処理開始前）を取得
    if num1 == 1:
    # 4 plaintext v=(v1,v2) 20×2=40decimal
        m1 = randint(10000000000000000000,99999999999999999999)
        m2 = randint(10000000000000000000,99999999999999999999)

    if num1 == 2:
    # 4 plaintext v=(v1,v2) 40×2=80decimal
        m1 = randint(1000000000000000000000000000000000000000,9999999999999999999999999999999999999999)
        m2 = randint(1000000000000000000000000000000000000000,9999999999999999999999999999999999999999)

    if num1 == 3: 
    # 4 plaintext v=(v1,v2) 80×2=160decimal
        m1 = randint(10000000000000000000000000000000000000000000000000000000000000000000000000000000,99999999999999999999999999999999999999999999999999999999999999999999999999999999)
        m2 = randint(10000000000000000000000000000000000000000000000000000000000000000000000000000000,99999999999999999999999999999999999999999999999999999999999999999999999999999999)

    # τ-message
    #if i == 1:
        # τ-message m=(m1,m2) 4×2=8decimal
        #m1 = randint(1000,9999)
        #m2 = randint(1000,9999)
        #print("τ-message:","(",m1,m2,")",len(str(m1))+len(str(m2)))
        
    decimal = len(str(m1))+len(str(m2))
    
    if i == 0:
        act = act +[m1,m2]
        print()
        print("Message m:")
        print("(",m1,m2,")",decimal,"decimal","(",int(decimal/0.30102999),"bit)")
        print()

    #if i == 1:
        #print("τ-message:","(",m1,m2,")")
        #print()

    # v0=ｖ+dv
    v01 = m1+dv1
    v02 = m2+dv2

    Det0 = 0
    B0 = 0
    B1 = 0
    B2 = 0
    while Det0 == 0 or B0 <= 1 or B1 <= 1 or B2 <= 1: # non-regular
        Tau = 1
        if num0 == 1: # 40decimal(128bit-sec.)        
            while len(str(Tau))-1 != 40 and len(str(Tau))-1 != 41 :
                Tau = Fraction(randint(-99999999999999999999,99999999999999999999),randint(10000000000000000000,99999999999999999999))
        elif num0 == 2: # 80decimal(256bit-sec.)
            while len(str(Tau))-1 != 80 and len(str(Tau))-1 != 81:
                Tau = Fraction(randint(-9999999999999999999999999999999999999999,9999999999999999999999999999999999999999),
                   randint(1000000000000000000000000000000000000000,9999999999999999999999999999999999999999)) 
        # print(Tau)
        T1 = 0
        T2 = 0
        bT = 0
        aT = []
        for j in range(3): # ct0(j = 0)、ct1(j = 1)、ct2(j = 2)
            # 平行移動　（Tauを固定の下で、T1,T2,T3　は各1個、値はすべて異なる）
            T = 0
            if j == 0:
                T = T + Fraction(randint(1000000,9999999),randint(1000000,9999999)) # β0　署名第１項d0作成の準備 
                T1 = T1 + T
                bT = bT + len(str(T.numerator))+len(str(T.denominator))
            if j == 1:
                T = T + Fraction(randint(1111000,9999900),randint(1000000,9999999)) # β1　署名第2項d1作成の準備
                while T1 >= T:
                    T = Fraction(randint(1111000,9999900),randint(1000000,9999999))
                T2 = T2 + T
                bT = bT + len(str(T.numerator))+len(str(T.denominator))
            if j == 2:
                T = T + Fraction(randint(1111110,9900000),randint(1000000,9999999)) # β2　署名第3項d2作成の準備
                while T2 >= T:
                    T = Fraction(randint(1111110,9900000),randint(1000000,9999999))
                bT = bT + len(str(T.numerator))+len(str(T.denominator))
                #print(T)
            if i == 0:
                aT = aT + [T]
            # print(T0,T1,T2)

            # encryption
            pk1 = 0 # first component
            pk2 = 0 # second component
            pk3 = 0 # third component
            # t2 in T:
            t2 = T
            pk11 =  (v01**2* (24* (-1 + Tau)**2 + 3* t2**6* (-1 + Tau)**2 - 2* t2**4* (-5 + (-14 + Tau)* Tau) + 16* t2**3 *(-2 + Tau + Tau**2)\
                - 8* t2**5* (-2 + Tau + Tau**2) + t2**2* (68 - 40* Tau + 44* Tau**2)) + v02**2* (24* (1 + Tau)**2 + 3* t2**6* (1 + Tau)**2 - 16\
                *t2**3 *(-1 + Tau + 2* Tau**2) + 8* t2**5* (-1 + Tau + 2* Tau**2) + t2**2* (44 + 40* Tau + 68* Tau**2) + 2* t2**4* (-1 + Tau* (\
                -14 + 5* Tau)))+ 2 *v01* v02* (24 - 24* Tau**2 + 4* t2**2* (5 + (6 - 5* Tau)* Tau) + 8* t2**3* (-1 + (-6 + Tau)* Tau) - 4* t2**5\
                * (-1 + (-6 + Tau)* Tau)- 3* t2**6* (-1 + Tau**2) + 2* t2**4* (-7 + Tau* (6 + 7* Tau))))
            pk12 = 64* t2**4
            pk1 = pk1 + Fraction(pk11,pk12)

            pk21 = -(2 *v01**3* (-1 + Tau)* (8 + t2**2* (-2 + t2**2)* (-4 + t2* (2 + t2)) - 2* (2 + t2)* (-2 + t2**2)* (-2 + t2 - t2**2 + t2**3)\
                * Tau + (8 - 4* t2**4 + t2**6)* Tau**2) + v02**3* (1 + Tau)* (56* (1 + Tau)**2 + 7* t2**6* (1 + Tau)**2 - 16* t2**3* (1 + Tau)* (\
                -1 + 3* Tau) + 8* t2**5* (1 + Tau)* (-1 + 3* Tau) - 2* t2**4* (21 + Tau* (34 + 19* Tau)) + 4* t2**2* (11 + Tau* (6 + 25* Tau)))\
                + v01**2* v02* (24* (-1 + Tau)**2* (1 + Tau) + 3* t2**6* (-1 + Tau)**2* (1 + Tau) + 8* t2**3* (-1 + Tau)* (5 + 3* Tau* (2 + Tau))\
                - 4* t2**5* (-1 + Tau)* (5 + 3* Tau* (2 + Tau)) + 2* t2**4* (-13 + Tau* (13 + 3* (1 - 5* Tau)* Tau)) + 4* t2**2* (27 + Tau* (9\
                + Tau* (11 + 13* Tau)))) - 4* v01* v02**2* (24* (-1 + Tau)* (1 + Tau)**2 + 3* t2**6* (-1 + Tau)* (1 + Tau)**2 - 2* t2**3* (1 + Tau)\
                * (-4 + 3* (-5 + Tau)* Tau) + t2**5* (1 + Tau)* (-4 + 3* (-5 + Tau)* Tau) + 2* t2**2* (-3 + Tau* (-18 + (-8 + Tau)* Tau)) + t2**4\
                * (15 + Tau* (14 - Tau* (19 + 14* Tau)))))
            pk22 = 64* t2**4* v01* v02* Tau**2
            pk2 = pk2 + Fraction(pk21,pk22)

            pk31 =  (2* v01**3* (1 + Tau)* (8 + t2**2* (-2 + t2**2)* (-4 + t2* (2 + t2)) - 2* (2 + t2)* (-2 + t2**2)* (-2 + t2 - t2**2 + t2**3)\
                * Tau + (8 - 4* t2**4 + t2**6)* Tau**2) + v02**3* (-1 + Tau)* (56* (1 + Tau)**2 + 7* t2**6* (1 + Tau)**2 - 16* t2**3* (1 + Tau)\
                * (-1 + 3* Tau) + 8* t2**5* (1 + Tau)* (-1 + 3* Tau) - 2* t2**4* (21 + Tau* (34 + 19* Tau)) + 4* t2**2* (11 + Tau* (6 + 25* Tau)))\
                - 4* v01* v02**2* (8* (1 + Tau)* (3 + Tau* (-8 + 3* Tau)) + t2**6* (1 + Tau)* (3 + Tau* (-8 + 3* Tau)) + t2**4* (-15 + Tau* (24 + 7\
                * (3 - 2* Tau)* Tau))+ 2* t2**2* (3 + Tau* (12 + (-18 + Tau)* Tau)) - 2* t2**3* (4 + Tau* (11 + Tau* (-22 + 3* Tau))) + t2**5* (4\
                + Tau* (11 + Tau* (-22 + 3* Tau)))) + v01**2* v02* (8* (-1 + Tau)* (3 + Tau* (-22 + 3* Tau)) + t2**6* (-1 + Tau)* (3 + Tau* (-22\
                + 3* Tau)) + 8* t2**3* (5 + Tau* (-13 + Tau + 3* Tau**2)) - 4* t2**5* (5 + Tau* (-13 + Tau + 3* Tau**2)) + 2* t2**4* (13 - 3* Tau\
                * (21 + Tau* (-19 + 5* Tau))) + 4* t2**2* (-27 + Tau* (37 + Tau* (-7 + 13* Tau)))))                
            pk32 = 64* t2**4* v01* v02* Tau**2
            pk3 = pk3 + Fraction(pk31,pk32)

            if j == 0:
                d0 = (pk1,pk2,pk3)

            if j == 1:
                d1 = (pk1,pk2,pk3)

            if j == 2:
                d2 = (pk1,pk2,pk3)
           
        # detection of regularity
        Det0 = Fraction(1,2)* (-d0[1]*d1[2]+d0[1]*d2[2]-d0[2]*d2[1]+d0[2]*d1[1]-d1[1]*d2[2]+d1[2]*d2[1]) 
        if Det0 !=0:
            A0000 = - (-88* (1 - Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1]\
            + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2]\
            + d2[1] + d2[2]))* Tau)**3 + 104* (1 - Tau)* Tau* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]\
            * (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2])\
            + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**3 - 88* (1 - Tau)* Tau**2* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]\
            * d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])\
            * d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**3 + 36* (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0]\
            + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0])\
            + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**2* (1 + Tau**2)* (d0[2]* d1[0] - d0[2]* d2[0]\
            - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0]\
            + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau)) + 22* (1 + Tau)* (d0[2]* d1[0]\
            - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0])\
            + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))**3 + 26* Tau\
            * (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])\
            + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0]\
            - d2[0])* (1 + Tau))**3 + 22* Tau**2* (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]\
            * (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1]\
            + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))**3 + 6* (1 - Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1]\
            - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0]\
            - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)* (-d0[2]* d1[0] + d0[2]* d2[0] + d1[1]* d2[0] - d1[2]* d2[0] - d1[0]\
            * d2[1] + d0[0]* (-d1[1] + d1[2] + d2[1] - d2[2]) + d1[0]* d2[2] + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]\
            * (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau + d0[1]* (d1[0] - d2[0])* (1 + Tau))**2* (5 + Tau* (13 + 5* Tau)))/(4* (-d1[1]* d2[0] + d0[1]* (-d1[0]\
            + d2[0]) + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau* (6* (-d1[1] + d1[2])* d2[0] + 6* d1[0]* (d2[1] - d2[2]) + 6* d0[0]* (d1[1] - d1[2] - d2[1]\
            + d2[2]) + ((d1[1] + 27* d1[2])* d2[0] - d1[0]* (d2[1] + 27* d2[2]) + d0[0]* (-d1[1] - 27* d1[2] + d2[1] + 27* d2[2]))* Tau + 3* ((d1[1] + d1[2])\
            * d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]* (d1[0] - d2[0])* (-6 + Tau + 3* Tau**2) + 3* d0[2]\
            * (d1[0] - d2[0])* (2 + Tau* (9 + Tau)))* (-3* (d0[2]* (d1[0] - d2[0]) - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1]\
            - d1[2] - d2[1] + d2[2])) + 2* (-d1[1]* d2[0] + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau + 3* (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0]\
            - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]* (d1[0] - d2[0])* (3 + Tau* (-2 + 3* Tau))))
            
            A0002= - ((96* d0[1]* (d0[2]* (d1[1] - d2[1]) + d1[2]* d2[1] -d1[1]* d2[2] + d0[1]* (-d1[2] + d2[2]))* Tau**4* (d0[2]* d1[0] - d0[2]* d2[0]\
            - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau)\
            + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] -d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)* (d0[2]* d1[0]\
            - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])\
            * d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))\
            + 14* (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])\
            + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0]\
            - d2[0])* (1 + Tau))**3 + Tau* (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1]\
            - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))\
            * Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))**3 + 23* Tau**2* (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1]\
            - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1]\
            - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))**3 - 4* (1 - Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0]\
            + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0]\
            - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**3* (14 + 5* Tau* (1 + Tau)) - 32* d0[0]\
            * (d0[2]* (d1[1] - d2[1]) + d1[2]* d2[1] - d1[1]* d2[2] + d0[1]* (-d1[2] + d2[2]))**2* Tau**4* (3* (d0[2]* (d1[0] - d2[0]) - d1[1]* d2[0] + d1[2]\
            * d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])) + 22* (-d1[1]* d2[0] + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau\
            + 3* (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau**2 - d0[1]* (d1[0]\
            - d2[0])* (3 + Tau* (22 + 3* Tau))) - 3* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1]\
            - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]\
            * (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)* (-d0[2]* d1[0] + d0[2]* d2[0] + d1[1]* d2[0] - d1[2]* d2[0] - d1[0]* d2[1] + d0[0]* (-d1[1] + d1[2]\
            + d2[1] - d2[2]) + d1[0]* d2[2] + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1]\
            + d2[2]))* Tau + d0[1]* (d1[0] - d2[0])* (1 + Tau))**2* (1 - Tau* (16 + Tau* (5 + 4* Tau))) + 6* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0]\
            + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0])\
            + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**2* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0]\
            + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]\
            * (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))* (17 + Tau* (7 + Tau* (13 + 11* Tau)))))/(4\
            * (-d1[1]* d2[0] + d0[1]* (-d1[0] + d2[0]) + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau* (6* (-d1[1] + d1[2])* d2[0] + 6* d1[0]* (d2[1] - d2[2])\
            + 6* d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + ((d1[1] + 27* d1[2])* d2[0] - d1[0]* (d2[1] + 27* d2[2]) + d0[0]* (-d1[1] - 27* d1[2] + d2[1] + 27\
            * d2[2]))* Tau + 3* ((d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]* (d1[0] - d2[0])\
            * (-6 + Tau + 3* Tau**2) + 3* d0[2]* (d1[0] - d2[0])* (2 + Tau* (9 + Tau)))* (-3* (d0[2]* (d1[0] - d2[0]) - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]\
            * d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])) + 2* (-d1[1]* d2[0] + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau + 3* (d0[2]\
            * (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]* (d1[0] - d2[0])\
            * (3 + Tau* (-2 + 3* Tau))))

            A1112 = - ((96* d1[1]* (d0[2]* (d1[1] - d2[1]) + d1[2]* d2[1] - d1[1]* d2[2] + d0[1]* (-d1[2] + d2[2]))* Tau**4* (d0[2]* d1[0] - d0[2]* d2[0]\
            - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau)\
            + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)* (d0[2]* d1[0]\
            - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0])\
            + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau)) + 14* (1\
            + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])\
            + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0]\
            - d2[0])* (1 + Tau))**3 + Tau* (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1]\
            - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))\
            * Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))**3 + 23* Tau**2* (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1]\
            - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1]\
            - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))**3 - 4* (1 - Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]\
            * d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0])\
            + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**3* (14 + 5* Tau* (1 + Tau)) + 32* d1[0]* (d0[2]\
            * (d1[1] - d2[1]) + d1[2]* d2[1] - d1[1]* d2[2] + d0[1]* (-d1[2] + d2[2]))**2* Tau**4* (-3* (d0[2]* (d1[0] - d2[0]) - d1[1]* d2[0] + d1[2]* d2[0]\
            + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])) + 22* (d1[1]* d2[0] - d1[0]* d2[1] + d0[0]* (-d1[1] + d2[1]))* Tau + 3\
            * (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]* (d1[0]\
            - d2[0])* (3 + Tau* (22 + 3* Tau))) - 3* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1]\
            - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]\
            * (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)* (-d0[2]* d1[0] + d0[2]* d2[0] + d1[1]* d2[0] - d1[2]* d2[0] - d1[0]* d2[1] + d0[0]* (-d1[1] + d1[2]\
            + d2[1] - d2[2]) + d1[0]* d2[2] + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1]\
            + d2[2]))* Tau + d0[1]* (d1[0] - d2[0])* (1 + Tau))**2* (1 - Tau* (16 + Tau* (5 + 4* Tau))) + 6* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0]\
            + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0]\
            - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**2* (d0[2]* d1[0] - d0[2]* d2[0]\
            - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0]\
            + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))* (17 + Tau* (7 + Tau* (13\
            + 11* Tau)))))/(4* (-d1[1]* d2[0] + d0[1]* (-d1[0] + d2[0]) + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau* (6* (-d1[1] + d1[2])* d2[0] + 6* d1[0]\
            * (d2[1] - d2[2]) + 6* d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + ((d1[1] + 27* d1[2])* d2[0] - d1[0]* (d2[1] + 27* d2[2]) + d0[0]* (-d1[1] - 27\
            * d1[2] + d2[1] + 27* d2[2]))* Tau + 3* ((d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]\
            * (d1[0] - d2[0])* (-6 + Tau + 3* Tau**2) + 3* d0[2]* (d1[0] - d2[0])* (2 + Tau* (9 + Tau)))* (-3* (d0[2]* (d1[0] - d2[0]) - d1[1]* d2[0] + d1[2]\
            * d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])) + 2* (-d1[1]* d2[0] + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau\
            + 3* (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]* (d1[0]\
            - d2[0])* (3 + Tau* (-2 + 3* Tau))))

            A2222 = - ((96* d2[1]* (d0[2]* (d1[1] - d2[1]) + d1[2]* d2[1] - d1[1]* d2[2] + d0[1]* (-d1[2] + d2[2]))* Tau**4* (d0[2]* d1[0] - d0[2]* d2[0]\
            - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]\
            * (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)* (d0[2]* d1[0] - d0[2]* d2[0]\
            - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0]\
            + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau)) + 14* (1 + Tau)* (d0[2]* d1[0]\
            - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0])\
            + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))**3 + Tau\
            * (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])\
            + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0]\
            - d2[0])* (1 + Tau))**3 + 23* Tau**2* (1 + Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]\
            * (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1]\
            + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))**3 - 4* (1 - Tau)* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1]\
            - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0]\
            - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**3* (14 + 5* Tau* (1 + Tau)) - 32* d2[0]* (d0[2]* (d1[1] - d2[1]) + d1[2]\
            * d2[1] - d1[1]* d2[2] + d0[1]* (-d1[2] + d2[2]))**2* Tau**4* (3* (d0[2]* (d1[0] - d2[0]) - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]\
            * d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])) + 22* (-d1[1]* d2[0] + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau + 3* (-((d1[1] + d1[2])* d2[0])\
            + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2]) + d1[0]* (d2[1] + d2[2]))* Tau**2 - d0[1]* (d1[0] - d2[0])* (3 + Tau* (22 + 3\
            * Tau))) - 3* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2])\
            + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1]\
            + d2[2]))* Tau)* (-d0[2]* d1[0] + d0[2]* d2[0] + d1[1]* d2[0] - d1[2]* d2[0] - d1[0]* d2[1] + d0[0]* (-d1[1] + d1[2] + d2[1] - d2[2]) + d1[0]* d2[2]\
            + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau + d0[1]* (d1[0] - d2[0])\
            * (1 + Tau))**2* (1 - Tau* (16 + Tau* (5 + 4* Tau))) + 6* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2]\
            + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + d0[1]* (d1[0] - d2[0])* (-1 + Tau) + (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0] - d1[0]* (d2[1]\
            + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau)**2* (d0[2]* d1[0] - d0[2]* d2[0] - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]\
            * d2[2] + d0[0]* (d1[1] - d1[2] - d2[1] + d2[2]) + (-((d1[1] + d1[2])* d2[0]) + d0[2]* (-d1[0] + d2[0]) + d0[0]* (d1[1] + d1[2] - d2[1] - d2[2])\
            + d1[0]* (d2[1] + d2[2]))* Tau - d0[1]* (d1[0] - d2[0])* (1 + Tau))* (17 + Tau* (7 + Tau* (13 + 11* Tau)))))/(4* (-d1[1]* d2[0] + d0[1]* (-d1[0]\
            + d2[0]) + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau* (6* (-d1[1] + d1[2])* d2[0] + 6* d1[0]* (d2[1] - d2[2]) + 6* d0[0]* (d1[1] - d1[2] - d2[1]\
            + d2[2]) + ((d1[1] + 27* d1[2])* d2[0] - d1[0]* (d2[1] + 27* d2[2]) + d0[0]* (-d1[1] - 27* d1[2] + d2[1] + 27* d2[2]))* Tau + 3* ((d1[1] + d1[2])\
            * d2[0] - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]* (d1[0] - d2[0])* (-6 + Tau + 3* Tau**2) + 3* d0[2]\
            * (d1[0] - d2[0])* (2 + Tau* (9 + Tau)))* (-3* (d0[2]* (d1[0] - d2[0]) - d1[1]* d2[0] + d1[2]* d2[0] + d1[0]* d2[1] - d1[0]* d2[2] + d0[0]* (d1[1]\
            - d1[2] - d2[1] + d2[2])) + 2* (-d1[1]* d2[0] + d0[0]* (d1[1] - d2[1]) + d1[0]* d2[1])* Tau + 3* (d0[2]* (d1[0] - d2[0]) + (d1[1] + d1[2])* d2[0]\
            - d1[0]* (d2[1] + d2[2]) + d0[0]* (-d1[1] - d1[2] + d2[1] + d2[2]))* Tau**2 + d0[1]* (d1[0] - d2[0])* (3 + Tau* (-2 + 3* Tau))))
            
            B0 = A0002 * A0000
            B1 = A1112 * A0000
            B2 = A2222 * A0000
            
    #print(Det0)
    #print(A0000)
    #print(A0002)
    #print(A1112)
    #print(A2222)
    #print()
    act = act + [d0[0]]+[d0[1]]+[d0[2]] + [d1[0]]+[d1[1]]+[d1[2]] + [d2[0]]+[d2[1]]+[d2[2]]
            
    #end = time.perf_counter()  # 現在時刻（処理完了後）を取得
    #time_diff = end - start  # 処理完了後の時刻から処理開始前の時刻を減算する

    # 暗号文総桁数、桁数リスト作成
    bl = len(str(d0+d1+d2))-14*9+2 # 暗号の総桁数(10進)
    #abl += bl
    #BL += [bl]

end = time.perf_counter()  # 現在時刻（処理完了後）を取得
time_diff = end - start  # 処理完了後の時刻から処理開始前の時刻を減算する
#print("署名作成時間",time_diff)  # 処理にかかった時間データを使用

# 文書署名ファイル化
with open('msig1.dat','wb') as file:
    pickle.dump(act,file)


# print("文書","m＝（",m1,m2,")")
print("Signature σ:")
print("σ＝（d0,d1,d2）:",bl,"decimal(",int(bl/0.30102999),"bit)")
# print("第１項　d0")
ds0 = []+[act[2]]+[act[3]]+[act[4]]
print("d0 =",ds0)
# print("第２項　d1")
ds1 = []+[act[5]]+[act[6]]+[act[7]]
print("d1 =",ds1)
# print("第３項　d2")
ds2 = []+[act[8]]+[act[9]]+[act[10]]
print("d2 =",ds2)
print()
print("τ:")
print(Tau,",",len(str(Tau))-1,"decimal")
print("制約値[β0,β1,β2]:")
print("[",str(aT[0]),",",str(aT[1]),",",str(aT[2]),"]",bT,"decimal(",int(bT/0.30102999),"bit)")
print(aT[0].numerator/aT[0].denominator,"<",aT[1].numerator/aT[1].denominator,"<",aT[2].numerator/aT[2].denominator)
#print(aT[0])
print()
print("署名作成時間",time_diff)  # 処理にかかった時間データを使用
#print(act)
#print(A0000)

Select number 1 or 2 (1:128bit-sec.,  2:256bit-sec.):  2
Select number 1 or 2 or 3 (1:40decimal(128bit), 2:80decimal(256bit), 3:160decimal(512bit)):  3



Message m:
( 41048022510904695094601200747686796898612324186984746433665686438262905850562032 63568392457294008515889462865488746631933616460358331106618463445917245637792266 ) 160 decimal ( 531 bit)

Signature σ:
σ＝（d0,d1,d2）: 5488 decimal( 18230 bit)
d0 = [Fraction(47613865838381271082767759336947092366600737253264617341772498590373889321692199874510562037124806825026979256854587049395075724781833662513071918038103132635739468810625139151553613213664470680096134884104642008286348216344797391974954895463854065852219971154817517333493730243, 33629187296639684038225047259082030195684152787808388299965689547164587348499032272088400277123297584506223161215287296), Fraction(-2117362456254986095114056563190378651212536673623226104209428097241636079982012714082497213218464082329614969158074594471875634334340102905247840241478409601227194627231262128660668843832997834765187783340836652106804165576352845798923825786551202981415750375908831538893686390771682408457268638135792124954768679945152